In [1]:
# import libraries
import datetime
import googlemaps
import itertools
import json
import numpy as np
import pandas as pd
import pymysql
import requests
import string
import time
import urllib
from bs4 import BeautifulSoup as bs
from config import gkey
from requests import get
from scipy import stats
from selenium import webdriver
from splinter import Browser
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()
from statsmodels.stats.proportion import proportions_ztest
from zodiac_sign import get_zodiac_sign

In [2]:
# Specify URL for scraping
url = "https://basketball.realgm.com/ncaa/players/"

# Create list of years to collect data from
years = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

# Create list of letters for all players with names beginning with those letters
alphabet = list(string.ascii_uppercase)

# Create Gmaps key linkage
gmaps = googlemaps.Client(key=gkey)

In [10]:
player_group = []
for year in years:
    for letter in alphabet:
        # Scrape pages into Soup for each year and letter of the alphabet
        page = urllib.request.urlopen(url+str(year)+"/"+letter)
        soup = bs(page, "html.parser")

        # Create a dictionary of characteristics for each player 
        players = soup.find_all("tr")
        iterplayers = iter(players)
        try:
            next(iterplayers)
        except Exception:
            pass
        for player in iterplayers:
            player_info = {}
            facts = player.find_all('td', {'class':'nowrap'})
            player_info['name'] = facts[0].text
            player_info['link'] = facts[0].a['href']
            player_info['pos'] = facts[1].text
            player_info['height'] = facts[2]['rel']
            player_info['weight'] = facts[3]['rel']
            player_info['college'] = facts[4]['rel']
            player_info['tenure'] = facts[5]['rel']
            player_info['dob'] = facts[6]['rel']
            player_info['pob'] = facts[7]['rel']
            player_info['high_school'] = facts[8]['rel']
            player_info['season'] = year

            # Add each player's characteristic dictionary to the player group list
            player_group.append(player_info)

In [14]:
# Create a dataframe using the player group list
ncaa_players = pd.DataFrame.from_records(player_group)

# See what columns were included
ncaa_players.columns

Index(['college', 'dob', 'height', 'high_school', 'link', 'name', 'pob', 'pos',
       'season', 'tenure', 'weight'],
      dtype='object')

In [15]:
# Check out dataframe
ncaa_players

,college,dob,height,high_school,link,name,pob,pos,season,tenure,weight
0,Arizona State,19880819,75,Desert Vista High School,/player/Ty-Abbott/Summary/6680,Ty Abbott,Phoenix (AZ),G,2009,4,209
1,UCLA,19880602,74,St. Louis Country Day School,/player/Mustafa-Abdul-Hamid/Summary/6888,Mustafa Abdul-Hamid,St. Louis (MO),G,2009,5,195
2,Mercer,19400101,69,N/A,/player/Larney-Abdush-Shakur/Summary/14491,Larney Abdush-Shakur,Austell (GA),PG,2009,4,153
3,Sacred Heart,19400101,77,Marianapolis Preparatory School,/player/Toso-Abedamowo/Summary/20808,Toso Abedamowo,Lincoln (RI),G,2009,2,190
4,Texas Southern,19870304,79,Creative Mind Foundation,/player/Onyekautukwu-Abiakam/Summary/15292,Onyekautukwu Abiakam,Lagos,F,2009,4,195
5,Brigham Young,19891104,77,Brewster Academy,/player/Charles-Abouo/Summary/9751,Charles Abouo,Abidjan,GF,2009,2,215
6,UCLA,19850102,81,Tilton School,/player/Alfred-Aboya/Summary/1736,Alfred Aboya,Yaounde,PF,2009,8,245
7,Appalachian State,19870609,70,Starmount High School,/player/Ryann-Abraham/Summary/20990,Ryann Abraham,Mayaguez (PR),G,2009,6,185
8,Texas,19861016,71,McNeil High School,/player/AJ-Abrams/Summary/1725,A.J. Abrams,Round Rock (TX),PG,2009,8,160
9,East Carolina,19890621,78,Highland Springs High School,/player/Jamar-Abrams/Summary/7705,Jamar Abrams,Richmond (VA),GF,2009,4,205


In [18]:
# Create a unique ID for each player
ncaa_players['name_dob'] = ncaa_players['name']+'_'+ncaa_players['dob']

In [20]:
ncaa_players

,college,dob,height,high_school,link,name,pob,pos,season,tenure,weight,name_dob
0,Arizona State,19880819,75,Desert Vista High School,/player/Ty-Abbott/Summary/6680,Ty Abbott,Phoenix (AZ),G,2009,4,209,Ty Abbott_19880819
1,UCLA,19880602,74,St. Louis Country Day School,/player/Mustafa-Abdul-Hamid/Summary/6888,Mustafa Abdul-Hamid,St. Louis (MO),G,2009,5,195,Mustafa Abdul-Hamid_19880602
2,Mercer,19400101,69,N/A,/player/Larney-Abdush-Shakur/Summary/14491,Larney Abdush-Shakur,Austell (GA),PG,2009,4,153,Larney Abdush-Shakur_19400101
3,Sacred Heart,19400101,77,Marianapolis Preparatory School,/player/Toso-Abedamowo/Summary/20808,Toso Abedamowo,Lincoln (RI),G,2009,2,190,Toso Abedamowo_19400101
4,Texas Southern,19870304,79,Creative Mind Foundation,/player/Onyekautukwu-Abiakam/Summary/15292,Onyekautukwu Abiakam,Lagos,F,2009,4,195,Onyekautukwu Abiakam_19870304
5,Brigham Young,19891104,77,Brewster Academy,/player/Charles-Abouo/Summary/9751,Charles Abouo,Abidjan,GF,2009,2,215,Charles Abouo_19891104
6,UCLA,19850102,81,Tilton School,/player/Alfred-Aboya/Summary/1736,Alfred Aboya,Yaounde,PF,2009,8,245,Alfred Aboya_19850102
7,Appalachian State,19870609,70,Starmount High School,/player/Ryann-Abraham/Summary/20990,Ryann Abraham,Mayaguez (PR),G,2009,6,185,Ryann Abraham_19870609
8,Texas,19861016,71,McNeil High School,/player/AJ-Abrams/Summary/1725,A.J. Abrams,Round Rock (TX),PG,2009,8,160,A.J. Abrams_19861016
9,East Carolina,19890621,78,Highland Springs High School,/player/Jamar-Abrams/Summary/7705,Jamar Abrams,Richmond (VA),GF,2009,4,205,Jamar Abrams_19890621


In [1]:
ncaa_players.loc[ncaa_players['name'] == "Kevin Love"]

NameError: name 'ncaa_players' is not defined

In [21]:
ncaa_players.to_csv("NCCA_Players.csv")